## Broadcasting

Broadcasting in NumPy refers to the ability to perform arithmetic operations on arrays of different shapes without explicitly copying data.
Instead of creating large intermediate arrays, NumPy stretches smaller arrays across larger ones so that element-wise operations can happen efficiently.

### Key Rules of Broadcasting
When operating on two arrays:


Compare their shapes from the trailing dimensions backward; <br>Dimensions are compatible if:
1. They are equal, OR
2. One of them is 1

If a dimension is 1, NumPy broadcasts (repeats) that dimension to match the other.

In [ ]:
import numpy as np 
def letter_vector_rec(size:int, vector:np.ndarray):
    if not len(vector):
        return np.array([f'{i}' for i in range(size)])
    else:
        # isn't this similar to kronecker product ?
        return np.array([f'{i}'+vector for i in range(size)])

def letter_vector(prefix:str, shape:list[int]):
    result = np.array([])
    for size in shape[::-1]:
        result = letter_vector_rec(size, result)
    return prefix + result

A = letter_vector('a_', shape=(2, 3, 4))
A

array([[['a_000', 'a_001', 'a_002', 'a_003'],
        ['a_010', 'a_011', 'a_012', 'a_013'],
        ['a_020', 'a_021', 'a_022', 'a_023']],

       [['a_100', 'a_101', 'a_102', 'a_103'],
        ['a_110', 'a_111', 'a_112', 'a_113'],
        ['a_120', 'a_121', 'a_122', 'a_123']]], dtype='<U5')

In [ ]:
B = letter_vector('b_', shape=(2, 1, 4))
B

array([[['b_000', 'b_001', 'b_002', 'b_003']],

       [['b_100', 'b_101', 'b_102', 'b_103']]], dtype='<U5')

In [25]:
A + B

array([[['a_000b_000', 'a_001b_001', 'a_002b_002', 'a_003b_003'],
        ['a_010b_000', 'a_011b_001', 'a_012b_002', 'a_013b_003'],
        ['a_020b_000', 'a_021b_001', 'a_022b_002', 'a_023b_003']],

       [['a_100b_100', 'a_101b_101', 'a_102b_102', 'a_103b_103'],
        ['a_110b_100', 'a_111b_101', 'a_112b_102', 'a_113b_103'],
        ['a_120b_100', 'a_121b_101', 'a_122b_102', 'a_123b_103']]],
      dtype='<U10')

### Explanation
B has been tiled across 2nd dimension first <br>
B's dimension is (2, 1, 4), after tiling it by repeating axes by (1, 3, 1) times, it becomes (2, 3, 4)

In [26]:
B_tiled = np.tile(B, reps=(1, 3, 1)) # 
B_tiled

array([[['b_000', 'b_001', 'b_002', 'b_003'],
        ['b_000', 'b_001', 'b_002', 'b_003'],
        ['b_000', 'b_001', 'b_002', 'b_003']],

       [['b_100', 'b_101', 'b_102', 'b_103'],
        ['b_100', 'b_101', 'b_102', 'b_103'],
        ['b_100', 'b_101', 'b_102', 'b_103']]], dtype='<U5')

In [27]:
A + B_tiled

array([[['a_000b_000', 'a_001b_001', 'a_002b_002', 'a_003b_003'],
        ['a_010b_000', 'a_011b_001', 'a_012b_002', 'a_013b_003'],
        ['a_020b_000', 'a_021b_001', 'a_022b_002', 'a_023b_003']],

       [['a_100b_100', 'a_101b_101', 'a_102b_102', 'a_103b_103'],
        ['a_110b_100', 'a_111b_101', 'a_112b_102', 'a_113b_103'],
        ['a_120b_100', 'a_121b_101', 'a_122b_102', 'a_123b_103']]],
      dtype='<U10')

In [28]:
np.all((A + B_tiled) == (A + B))

np.True_

### Invalid cases
Broadcasting does not work with invalid dimensions

In [30]:
C = letter_vector('c_', shape=(2, 3))
C

array([['c_00', 'c_01', 'c_02'],
       ['c_10', 'c_11', 'c_12']], dtype='<U4')

In [31]:
A + C

ValueError: operands could not be broadcast together with shapes (2,3,4) (2,3) 

In [35]:
D = letter_vector('d_', shape=(2, 2, 4))
D

array([[['d_000', 'd_001', 'd_002', 'd_003'],
        ['d_010', 'd_011', 'd_012', 'd_013']],

       [['d_100', 'd_101', 'd_102', 'd_103'],
        ['d_110', 'd_111', 'd_112', 'd_113']]], dtype='<U5')

Broadcasting does not work where shapes cannot be matched by tiling 

In [ ]:
# this wont work,
# as we can't tile D (2, 2, 4) on dimension=1 to match shape with A (2, 3, 4)
# this is because D[dimension=1]=2 and it cannot be multiplied by a constant to match A[dimension=1] which is 3

A + D

ValueError: operands could not be broadcast together with shapes (2,3,4) (2,2,4) 

### Mathemtical representation of tiling

* tiling of an array along dimension d<sub>i</sub> by *r* repetitions is equivalent to kronecker product of
tiling matrix and itself; A tiling matrix is an array of same dimension as A, but size=1 in all dimensions except size=r in the dimension of tiling.

* tiling of array by repetitions in each dimension given by t_r=(r1, r2 ...) is equal to kronecker product of tiling matrix of shape=t_r and itself

i.e;

$$
tile(A, reps=(r_1, r_2, r_3 ...)) = 1_{(r_1, r_2, r_3..)} \otimes A
$$


In [47]:
A_numeric = letter_vector('', (3, 3)).astype(np.int32)
A_numeric

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22]], dtype=int32)

In [49]:
np.tile(A_numeric, reps=(1, 3))

array([[ 0,  1,  2,  0,  1,  2,  0,  1,  2],
       [10, 11, 12, 10, 11, 12, 10, 11, 12],
       [20, 21, 22, 20, 21, 22, 20, 21, 22]], dtype=int32)

In [59]:
I = np.ones((1, 3), dtype=np.int32)
I

array([[1, 1, 1]], dtype=int32)

In [60]:
np.kron(I, A_numeric)

array([[ 0,  1,  2,  0,  1,  2,  0,  1,  2],
       [10, 11, 12, 10, 11, 12, 10, 11, 12],
       [20, 21, 22, 20, 21, 22, 20, 21, 22]], dtype=int32)

In [61]:
np.all(
    np.kron(I, A_numeric) == np.tile(A_numeric, reps=(1, 3))
)

np.True_

In [62]:
np.kron(I.T, A_numeric)

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22]], dtype=int32)

In [63]:
np.all(
    np.kron(I.T, A_numeric) == np.tile(A_numeric, reps=(3, 1))
)

np.True_

### Mathematical representation of broadcasting
Assume an operation f on 2 ndarrays, A of shape S<sub>a</sub> and B of shape S<sub>b</sub> uses broadcasting, then 

$$
f(A_{s_a}, B_{s_b}) = f((1_{K(s_a,s_b)} \otimes A ) , (1_{K(s_b,s_a)} \otimes B))
$$
 
$$
k(a, b) =
\begin{cases}
a, & \text{if } a = b \\
a, & \text{if } b = 1 \\
b, & \text{if } a = 1 \\
error, & \text{else}
\end{cases}
$$

$$
K(s_a, s_b) = (k(a_1, b_1), k(a_2, b_2) ...)
$$

⊗ is kronecker product.